In [1]:
import sys
sys.path.append("..")
from azure.cosmos import CosmosClient
from shared_utilities import helpers

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# Change fuzzy match to sql join

# Do I even need fuzzy matching ?????


In [3]:
conn_string = helpers.read_json("../local.settings.json")["Values"]["cosmosdb_deets"]
storageClient = CosmosClient.from_connection_string(conn_str=conn_string)

In [4]:
next_saturday, next_sunday, next_monday = helpers.find_next_weekend()
# change second input '{next_saturday} and {next_sunday}' in the query below.


# get Betfair 
query = f"SELECT * FROM c where STARTSWITH(c.openDate, '2024-02-24') or STARTSWITH(c.openDate, '2024-02-25')" 
betfair_list = []
database = storageClient.get_database_client("fightstore")
betfair_mma_events = database.get_container_client("betfair_mma_events")
cosmos_bf = betfair_mma_events.query_items(query,enable_cross_partition_query= True)
for item in cosmos_bf:
    betfair_list.append(item)
         
# get Cards            
query = f"SELECT * FROM c where STARTSWITH(c.cardDate, '2024-02-24') or STARTSWITH(c.cardDate, '2024-02-25')"
cards=[]
fc_mma_cards = database.get_container_client("fc_mma_cards")
cosmos_fc = fc_mma_cards.query_items(query,enable_cross_partition_query = True)
for idc, card in enumerate(cosmos_fc):
    for idf, fight in enumerate(card["fights"]):
        card["fights"][idf]["index"]=idf # add the index to the specific fight object for patch later
    cards.append(card)


In [5]:
cards

[{'title': 'RIZIN LANDMARK 8 in SAGA',
  'date': 'Saturday, February 24, 12:00 AM ET',
  'link': 'https://www.tapology.com/fightcenter/events/109005-rizin-landmark-8-in-saga',
  'fights': [{'main': True,
    'weight': '157',
    'fighterA': {'name': 'Luiz Gustavo',
     'record': '12-2-0',
     'country': 'https://www.tapology.com/assets/flags/BR-45af1ab77cd750eff617a8b71f64b318c9a4ecf06c863a2c63cff71550930fe7.gif',
     'picture': 'https://images.tapology.com/headshot_images/93025/icon/killer2.jpg?1511318028',
     'link': 'https://www.tapology.com/fightcenter/fighters/93025-luiz-gustavo-killer'},
    'fighterB': {'name': 'Yoshinori Horie',
     'record': '12-3-0',
     'country': 'https://www.tapology.com/assets/flags/JP-9a8638bb4ad16191d186a0ecfb4a896adc79ecd2829785ed00e32d6537ae88e3.gif',
     'picture': 'https://images.tapology.com/headshot_images/137641/icon/Yoshinori_Horie.jpg?1562264488',
     'link': 'https://www.tapology.com/fightcenter/fighters/137641-yoshinori-horie'},
    

In [6]:
matched_cards, not_on_betfair = helpers.fuzzy_match_dict_list(cards, betfair_list, "fight_name" ,"name")

In [7]:
matched_cards # log these 

[{'title': 'RIZIN LANDMARK 8 in SAGA',
  'date': 'Saturday, February 24, 12:00 AM ET',
  'link': 'https://www.tapology.com/fightcenter/events/109005-rizin-landmark-8-in-saga',
  'updatedAt': 'https://www.tapology.com/fightcenter/events/109005-rizin-landmark-8-in-saga',
  'id': '2253205469872132028',
  'cardDate': '2024-02-24',
  'partition': '2324851694792783257',
  'fights': [{'main': True,
    'weight': '126',
    'fighterA': {'name': 'Yuki Ito',
     'record': '15-5-0',
     'country': 'https://www.tapology.com/assets/flags/JP-9a8638bb4ad16191d186a0ecfb4a896adc79ecd2829785ed00e32d6537ae88e3.gif',
     'picture': 'https://images.tapology.com/headshot_images/131333/icon/Untitleff.jpg?1563024477',
     'link': 'https://www.tapology.com/fightcenter/fighters/131333-yuki-ito'},
    'fighterB': {'name': 'Masatoshi Ueda',
     'record': '19-9-2',
     'country': 'https://www.tapology.com/assets/flags/JP-9a8638bb4ad16191d186a0ecfb4a896adc79ecd2829785ed00e32d6537ae88e3.gif',
     'picture': '

In [8]:
not_on_betfair # -> These are fights in the card which we can't find in betfair

[{'main': True,
  'weight': '157',
  'fighterA': {'name': 'Luiz Gustavo',
   'record': '12-2-0',
   'country': 'https://www.tapology.com/assets/flags/BR-45af1ab77cd750eff617a8b71f64b318c9a4ecf06c863a2c63cff71550930fe7.gif',
   'picture': 'https://images.tapology.com/headshot_images/93025/icon/killer2.jpg?1511318028',
   'link': 'https://www.tapology.com/fightcenter/fighters/93025-luiz-gustavo-killer'},
  'fighterB': {'name': 'Yoshinori Horie',
   'record': '12-3-0',
   'country': 'https://www.tapology.com/assets/flags/JP-9a8638bb4ad16191d186a0ecfb4a896adc79ecd2829785ed00e32d6537ae88e3.gif',
   'picture': 'https://images.tapology.com/headshot_images/137641/icon/Yoshinori_Horie.jpg?1562264488',
   'link': 'https://www.tapology.com/fightcenter/fighters/137641-yoshinori-horie'},
  'fight_name': 'Luiz Gustavo v Yoshinori Horie',
  'fight_id': '7595981172308424702',
  'index': 0},
 {'main': True,
  'weight': '146',
  'fighterA': {'name': 'Vugar Karamov',
   'record': '19-5-0',
   'country': 

In [10]:
# not going to work because it removes all over fights.....

for card in matched_cards:
        # print(card)
        for fight in card["fights"]:
            # print(fight)
            # print(fight["index"])
            operations =[
                { "op": "add", "path": "/fights/"+str(fight["index"])+"/betfair_event_id", "value": fight["betfair_event_id"] },
                { "op": "add", "path": "/fights/"+str(fight["index"])+"/betfair_country_code", "value": fight["betfair_country_code"] },
                { "op": "add", "path": "/fights/"+str(fight["index"])+"/betfair_open_date", "value": fight["betfair_open_date"] },
                { "op": "add", "path": "/fights/"+str(fight["index"])+"/betfair_timezone", "value": fight["betfair_timezone"] }]

        print(operations)
        # response = fc_mma_cards.patch_item(item=card["id"], partition_key=card["link"], patch_operations=operations)

 
    # only upsert because fight will have existed beforehand.
    
    # no need to pure create because it will have been created from ingestion
    # no need to patch because this will be the truth now..

[{'op': 'add', 'path': '/fights/13/betfair_event_id', 'value': '32988781'}, {'op': 'add', 'path': '/fights/13/betfair_country_code', 'value': 'GB'}, {'op': 'add', 'path': '/fights/13/betfair_open_date', 'value': '2024-02-25T02:25:00.000Z'}, {'op': 'add', 'path': '/fights/13/betfair_timezone', 'value': 'GMT'}]
[{'op': 'add', 'path': '/fights/12/betfair_event_id', 'value': '32988786'}, {'op': 'add', 'path': '/fights/12/betfair_country_code', 'value': 'GB'}, {'op': 'add', 'path': '/fights/12/betfair_open_date', 'value': '2024-02-25T00:10:00.000Z'}, {'op': 'add', 'path': '/fights/12/betfair_timezone', 'value': 'GMT'}]


In [15]:
"NOT ON BETFAIR: "+ str(not_on_betfair)

"NOT ON BETFAIR: [{'main': True, 'weight': '157', 'fighterA': {'name': 'Luiz Gustavo', 'record': '12-2-0', 'country': 'https://www.tapology.com/assets/flags/BR-45af1ab77cd750eff617a8b71f64b318c9a4ecf06c863a2c63cff71550930fe7.gif', 'picture': 'https://images.tapology.com/headshot_images/93025/icon/killer2.jpg?1511318028', 'link': 'https://www.tapology.com/fightcenter/fighters/93025-luiz-gustavo-killer'}, 'fighterB': {'name': 'Yoshinori Horie', 'record': '12-3-0', 'country': 'https://www.tapology.com/assets/flags/JP-9a8638bb4ad16191d186a0ecfb4a896adc79ecd2829785ed00e32d6537ae88e3.gif', 'picture': 'https://images.tapology.com/headshot_images/137641/icon/Yoshinori_Horie.jpg?1562264488', 'link': 'https://www.tapology.com/fightcenter/fighters/137641-yoshinori-horie'}, 'fight_name': 'Luiz Gustavo v Yoshinori Horie', 'fight_id': '7595981172308424702', 'index': 0}, {'main': True, 'weight': '146', 'fighterA': {'name': 'Vugar Karamov', 'record': '19-5-0', 'country': 'https://www.tapology.com/asse

In [68]:
response # log this.

[{'op': 'add', 'path': '/fights/0/betfair_event_id', 'value': '32834431'},
 {'op': 'add', 'path': '/fights/0/betfair_country_code', 'value': 'GB'},
 {'op': 'add',
  'path': '/fights/0/betfair_open_date',
  'value': '2024-02-18T05:00:00.000Z'},
 {'op': 'add', 'path': '/fights/0/betfair_timezone', 'value': 'GMT'}]

In [66]:
# ops = [{ "op": "add", "path": "/fights/0/test", "value": "patrick"}]
# response = fc_mma_cards.patch_item(item="4797802081340855333", partition_key="https://www.tapology.com/fightcenter/events/107060-ufc-298", patch_operations=ops)


{'title': 'UFC 298',
 'date': 'Saturday, February 17,  6:30 PM ET',
 'link': 'https://www.tapology.com/fightcenter/events/107060-ufc-298',
 'fights': [{'main': True,
   'weight': '145',
   'fighterA': {'name': 'Alexander Volkanovski',
    'record': '26-3-0',
    'country': 'https://www.tapology.com/assets/flags/AU-429ed070b76f87de04728ef13b9d68e308be36cf4444bb32eb97564ae7016ff1.gif',
    'picture': 'https://images.tapology.com/headshot_images/41705/icon/alexander-volkanovski.jpg?1567271247',
    'link': 'https://www.tapology.com/fightcenter/fighters/41705-alex-volkanski'},
   'fighterB': {'name': 'Ilia Topuria',
    'record': '14-0-0',
    'country': 'https://www.tapology.com/assets/flags/GE-46e71ebd13f4e3f14b56f61538a1d026361bee9e3a68ed169cf30520d087c2fa.gif',
    'picture': 'https://images.tapology.com/headshot_images/129278/icon/Topuria-Hero.jpg?1659102670',
    'link': 'https://www.tapology.com/fightcenter/fighters/129278-ilia-topuria'},
   'fight_name': 'Alexander Volkanovski v Il

In [ ]:
# Work out where to store this. 
# schema enforcement.
# make date more reliable.
# automate get bf api session token
# cosmos data model
    # how to add specific schemas.

In [37]:
import json
operations

[{'op': 'add', 'path': 'fights/0/betfair_event_id', 'value': '32834431'},
 {'op': 'add', 'path': 'fights/0/betfair_country_code', 'value': 'GB'},
 {'op': 'add',
  'path': 'fights/0/betfair_open_date',
  'value': '2024-02-18T05:00:00.000Z'},
 {'op': 'add', 'path': 'fights/0/betfair_timezone', 'value': 'GMT'}]